<a href="https://colab.research.google.com/github/gouthamans14/deeplearning-tesnorflow/blob/main/NLP/NLP_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundementals using Tensorflow

In [89]:
! nvidia-smi

Tue Jul  4 06:09:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W /  70W |    421MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [90]:
!wget https://raw.githubusercontent.com/gouthamans14/deeplearning-tesnorflow/main/extras/helper_functions.py

--2023-07-04 06:09:23--  https://raw.githubusercontent.com/gouthamans14/deeplearning-tesnorflow/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.2’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-04 06:09:23 (73.2 MB/s) - ‘helper_functions.py.2’ saved [10246/10246]



In [91]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

### Get a text dataset


In [92]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-07-04 06:09:23--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.103.128, 108.177.120.128, 142.251.171.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.103.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.2’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-07-04 06:09:23 (86.3 MB/s) - ‘nlp_getting_started.zip.2’ saved [607343/607343]



### Visualizing a text data

In [93]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [94]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [95]:
train_df[train_df["target"]==0]

,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0
17,25,NaN,NaN,Summer is lovely,0
18,26,NaN,NaN,My car is so fast,0
19,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0
...,...,...,...,...,...
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0
7582,10834,wrecked,NaN,Cramer: Iger's 3 words that wrecked Disney's s...,0
7584,10837,NaN,NaN,These boxes are ready to explode! Exploding Ki...,0
7587,10841,NaN,NaN,Sirens everywhere!,0


In [96]:
train_df_shuffled = train_df.sample(random_state=42, frac=1) #frac - what percent of data in percentage to be shuffled 1 - 100%

In [97]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [98]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [99]:
100*(train_df.target.value_counts()/train_df.shape[0]) # Almost balanced

0    57.034021
1    42.965979
Name: target, dtype: float64

In [100]:
len(train_df), len(test_df)

(7613, 3263)

In [101]:
# Lets visualize some random samples

import random

random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target: {target}")
  print(f"Real disaster" if target>0 else "Not real disaster")
  print(f"Text: \n {text}\n")

Target: 0
Not real disaster
Text: 
 IF ANYONE WERE TO HARM THE BOYS THEY WOULD GET TAKEN DOWN IMMEDIATELY NOT BY SECURITY BUT BY THE FANS REAL QUICK

Target: 1
Real disaster
Text: 
 #SanDiego #News Sinkhole Disrupts Downtown Trolley Service: The incident happened Wed... http://t.co/RVMMuT3GvC #Algeria #???????

Target: 1
Real disaster
Text: 
 Jacksonville Fire &amp; Rescue Engine 58 - AWESOME TRUCK! - OFFICIAL JADE FL... https://t.co/ybdSlJw7c1 via @YouTube

Target: 0
Not real disaster
Text: 
 how will I survive without dorrian

Target: 1
Real disaster
Text: 
 Governor Allows Parole for School Bus Hijacker http://t.co/u4bdy1W7d4



In [102]:
from sklearn.model_selection import train_test_split


In [103]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) #

In [104]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                           train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [105]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

### Converting text to numbers
In NLP, there are two main concepts for turning text into numbers:

* Tokenization - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
1) Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.<br>

2) Character-level* tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.<br>

3) Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.<br>

* *Embeddings* - An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use embeddings:<br>
<br>
1) Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.<br>
2) Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

### Text Vectorization

In [106]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [107]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [108]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [109]:
#Use the default TextVectorization Parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                  split ="whitespace",
                                    ngrams=None,
                                    output_mode = "int",  #it can also be tf-idf
                                    output_sequence_length = None,
                                   # pad_to_max_tokens=True
                                    )

In [110]:
round(sum([len(i.split()) for i in train_sentences])/train_sentences.shape[0])

15

In [111]:
max_vocab_length = 10000
max_length =15

text_vectorizer = TextVectorization(max_tokens =max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length =max_length)

In [112]:
#Fit the text vecotrizer to the training data
text_vectorizer.adapt(train_sentences)

In [113]:
sample_sentence ="There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [114]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 This Attempted Mass Murder brought to You by the Obama Administration http://t.co/jbrK8ZsrY6       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19,    1,  157,  315, 1545,    5,   12,   18,    2,  213, 6263,
           1,    0,    0,    0]])>

In [115]:
# Get this unique wprds int the vocab

words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [116]:
len(words_in_vocab)

10000

In [117]:
top_5_words

['', '[UNK]', 'the', 'a', 'in']

In [118]:
bottom_5_words

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

### Creating an Embedding using an Embedding Layer

Parameters in an embedding layer:

* Input_dim = The size of our vocab
* Output_dim = The size of the outpur embedding vector for example a value  of100 mean each token gets represented by a 100 vector long
* Input Length = Length of the sequences passed to embedding layer
*

In [119]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim =128,
                             embeddings_initializer = "uniform",
                             input_length = max_length,
                             )

In [120]:
embedding

In [121]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text: \n {random_sentence}")
print("\n\nEmbedded Version: \n")
embedding(text_vectorizer([random_sentence]))

Original Text: 
 @SwellyJetEvo Disneyland! Tacos there are bomb!


Embedded Version: 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00678349, -0.03436315, -0.00839504, ..., -0.00716221,
         -0.04765108,  0.02598638],
        [ 0.01922962,  0.04481827,  0.0005817 , ..., -0.02024578,
         -0.04650973,  0.01062776],
        [ 0.04503666,  0.00993271, -0.04790157, ..., -0.01999954,
         -0.04671525, -0.01271225],
        ...,
        [ 0.02047693,  0.0195818 , -0.00791135, ..., -0.04646126,
         -0.03480067, -0.00368891],
        [ 0.02047693,  0.0195818 , -0.00791135, ..., -0.04646126,
         -0.03480067, -0.00368891],
        [ 0.02047693,  0.0195818 , -0.00791135, ..., -0.04646126,
         -0.03480067, -0.00368891]]], dtype=float32)>

### Mode 0: Baseline Model

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [123]:
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

# Fit the pipeline to the trainig data

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [124]:
baseline_score = model_0.score(val_sentences,val_labels)

In [125]:
baseline_score*100

79.26509186351706

In [126]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [127]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [128]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A Simple Dense Model

In [129]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"


In [130]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype =tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs, name="Model_1_dense")


In [131]:
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 dense_2 (Dense)             (None, 15, 1)             129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [132]:
model_1.compile(loss=tf.losses.BinaryCrossentropy(),
                optimizer = tf.optimizers.Adam(),
                metrics=["accuracy"])

model_1.fit(x =train_sentences, y = train_labels,
            epochs=5,
            validation_data =(val_sentences,val_labels),
            callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                    "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230704-060924
Epoch 1/5


ValueError: ignored

In [133]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [134]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [135]:
model_1.compile(loss=tf.losses.BinaryCrossentropy(),
                optimizer = tf.optimizers.Adam(),
                metrics=["accuracy"])

model_1.fit(x =train_sentences, y = train_labels,
            epochs=5,
            validation_data =(val_sentences,val_labels),
            callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                    "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230704-060933
Epoch 1/5
215/215 [==============================] - 22s 76ms/step - loss: 0.6123 - accuracy: 0.6866 - val_loss: 0.5360 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4414 - accuracy: 0.8180 - val_loss: 0.4734 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3466 - accuracy: 0.8600 - val_loss: 0.4574 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4624 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2379 - accuracy: 0.9120 - val_loss: 0.4826 - val_accuracy: 0.7887


In [136]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4826 - accuracy: 0.7887


[0.48255935311317444, 0.7887139320373535]

In [137]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

### Model 2: LSTM

Our structure of RNN looks like this :

Input (text) --> Tokenize --> Embedding --> Layer --> Output

In [138]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64,return_sequences=True)(x) #When u are stacking RNN cells together we need to return sequences.
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
x = layers.Dense(64,activation="relu" )(x)
#print(x.shape)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs, name="Model_2_LSTM")




In [139]:
model_2.summary()

Model: "Model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                      

In [140]:
model_2.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics =["accuracy"])


In [141]:
from sklearn import metrics
model_2_history = model_2.fit(x = train_sentences, y = train_labels,
                              epochs = 5 ,
                              validation_data =(val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "model_2_LSTM")])



Saving TensorBoard log files to: model_logs/model_2_LSTM/20230704-061300
Epoch 1/5
215/215 [==============================] - 20s 75ms/step - loss: 0.2224 - accuracy: 0.9242 - val_loss: 0.5395 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.1559 - accuracy: 0.9407 - val_loss: 0.6845 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1249 - accuracy: 0.9521 - val_loss: 0.6627 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1032 - accuracy: 0.9596 - val_loss: 0.7382 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0854 - accuracy: 0.9663 - val_loss: 1.0034 - val_accuracy: 0.7835


In [142]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[5.8496553e-03],
       [7.9105520e-01],
       [9.9994314e-01],
       [3.9202355e-02],
       [1.8416154e-04],
       [9.9959749e-01],
       [9.2547601e-01],
       [9.9997890e-01],
       [9.9992847e-01],
       [3.8207680e-01]], dtype=float32)

In [143]:
model_2_pred =tf.squeeze(tf.round(model_2_pred_probs))

In [144]:
model_2_results = calculate_results(y_true= val_labels,
                                    y_pred = model_2_pred
                                    )

In [145]:
model_2_results

{'accuracy': 78.34645669291339,
 'precision': 0.7845538066180612,
 'recall': 0.7834645669291339,
 'f1': 0.7819078902760987}

### Model 3 : GRU

In [146]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64,activation="relu")(x)
outputs= layers.Dense(1,activation="sigmoid")(x)

model_3 =tf.keras.Model(inputs,outputs)

In [147]:
model_3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                             

In [148]:
model_3.compile(loss ="binary_crossentropy",
                metrics=["accuracy"],
                optimizer =tf.keras.optimizers.Adam())

In [149]:
model_3_history = model_3.fit(x=train_sentences,y=train_labels,epochs=5,
              validation_data = (val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230704-061344
Epoch 1/5
215/215 [==============================] - 17s 59ms/step - loss: 0.1566 - accuracy: 0.9428 - val_loss: 0.8488 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0825 - accuracy: 0.9696 - val_loss: 1.0575 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0665 - accuracy: 0.9743 - val_loss: 1.0250 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0507 - accuracy: 0.9777 - val_loss: 1.2244 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0503 - accuracy: 0.9774 - val_loss: 1.4023 - val_accuracy: 0.7769


In [150]:
model_3.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 7ms/step - loss: 1.4023 - accuracy: 0.7769


[1.40228271484375, 0.7769029140472412]

In [151]:
model_3_pred_prob = model_3.predict(val_sentences)
model_3_preds = tf.squeeze(tf.round(model_3_pred_prob))

24/24 [==============================] - 1s 5ms/step


In [152]:
model_3_results = calculate_results(y_true = val_labels,
                                    y_pred = model_3_preds)

In [153]:
model_3_results

{'accuracy': 77.69028871391076,
 'precision': 0.7771036524564976,
 'recall': 0.7769028871391076,
 'f1': 0.7757784765658782}

### Model 4: Bidirectional

In [154]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
#x = layers.GlobalAveragePooling()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs,outputs)
model_4.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [155]:
model_4.compile(loss="binary_crossentropy",
                optimizer ="Adam",
                metrics =["accuracy"])

In [156]:
model_4_hitosry = model_4.fit(train_sentences,train_labels, epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230704-061414
Epoch 1/5
215/215 [==============================] - 24s 73ms/step - loss: 0.1008 - accuracy: 0.9658 - val_loss: 0.9295 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0566 - accuracy: 0.9755 - val_loss: 1.2488 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0457 - accuracy: 0.9780 - val_loss: 1.3845 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0395 - accuracy: 0.9812 - val_loss: 1.4876 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0407 - accuracy: 0.9781 - val_loss: 1.5092 - val_accuracy: 0.7651


In [157]:
model_4_predict_prob = model_4.predict(val_sentences)
model_4_pred = tf.squeeze(tf.round(model_4_predict_prob))

24/24 [==============================] - 2s 4ms/step


In [158]:
model_4_results = calculate_results(y_pred = model_4_pred,
                                    y_true=val_labels)

In [159]:
model_4_results

{'accuracy': 76.50918635170603,
 'precision': 0.7654719672276178,
 'recall': 0.7650918635170604,
 'f1': 0.7636361523198555}

### Convulutional Neural Network for Text

In [160]:
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["This is a test sentence"]))
conv_1d = layers.Conv1D(filters = 32, kernel_size =5, strides=1,activation="relu",
                        padding ="same")(embedding_test)
#conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d)

embedding_test.shape, conv_1d.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

### PreTrained Model

In [161]:
import tensorflow_hub as hub

In [162]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [173]:
sentence_encode_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",input_shape=[],
                                       dtype="string",
                                       trainable=False,
                                       name="USE")

In [193]:
model_6 = tf.keras.Sequential([
    sentence_encode_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_use")

model_6.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])
model_6.fit(train_sentences,train_labels,
            epochs=5,
            validation_data = (val_sentences,val_labels)
         )

Epoch 1/5
215/215 [==============================] - 5s 16ms/step - loss: 0.5024 - accuracy: 0.7846 - val_loss: 0.4515 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4151 - accuracy: 0.8151 - val_loss: 0.4422 - val_accuracy: 0.8005
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4018 - accuracy: 0.8209 - val_loss: 0.4375 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3931 - accuracy: 0.8262 - val_loss: 0.4311 - val_accuracy: 0.8202
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3885 - accuracy: 0.8279 - val_loss: 0.4269 - val_accuracy: 0.8189


In [194]:
model_6.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 13ms/step - loss: 0.4269 - accuracy: 0.8189


[0.4268721044063568, 0.8188976645469666]

In [195]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

24/24 [==============================] - 1s 11ms/step


In [260]:
model_6_results= calculate_results(model_6_preds,val_labels)

In [261]:
model_6_results

{'accuracy': 81.88976377952756,
 'precision': 0.8223222159698802,
 'recall': 0.8188976377952756,
 'f1': 0.8196547366984069}

### Model 7 : 10% Data



Transferlearning helps when you don't have a large dataset.

Experimenting with 10percent dataset

In [264]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [271]:
train_10_percent = train_df_shuffled[["text","target"]].sample(frac=0.1,random_state=42)


In [197]:
train_10_percent.head()

,text,target
4955,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0
584,FedEx no longer to transport bioterror germs i...,0
7411,Gunmen kill four in El Salvador bus attack: Su...,1
5950,@camilacabello97 Internally and externally scr...,1
5541,Radiation emergency #preparedness starts with ...,1


In [200]:
len(train_df_shuffled)

7613

In [199]:
train_sentences_10_percent = train_10_percent["text"]
train_labels_10_percent = train_10_percent["target"]

len(train_sentences_10_percent), len(train_labels_10_percent)


(761, 761)

In [203]:
model_7 = tf.keras.models.clone_model(model_6)
# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

Model: "model_6_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_14 (Dense)            (None, 64)                32832     
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [204]:
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_tf_hub_sentence_encoder")])


Saving TensorBoard log files to: model_logs/10_percent_tf_hub_sentence_encoder/20230704-073916
Epoch 1/5
24/24 [==============================] - 5s 45ms/step - loss: 0.6687 - accuracy: 0.6478 - val_loss: 0.6265 - val_accuracy: 0.7822
Epoch 2/5
24/24 [==============================] - 1s 27ms/step - loss: 0.5924 - accuracy: 0.7740 - val_loss: 0.5479 - val_accuracy: 0.7913
Epoch 3/5
24/24 [==============================] - 1s 28ms/step - loss: 0.5188 - accuracy: 0.7989 - val_loss: 0.4821 - val_accuracy: 0.8136
Epoch 4/5
24/24 [==============================] - 1s 28ms/step - loss: 0.4649 - accuracy: 0.8068 - val_loss: 0.4379 - val_accuracy: 0.8163
Epoch 5/5
24/24 [==============================] - 1s 27ms/step - loss: 0.4290 - accuracy: 0.8147 - val_loss: 0.4083 - val_accuracy: 0.8281


In [255]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

24/24 [==============================] - 0s 11ms/step


In [258]:
model_7_results = calculate_results(model_7_preds,val_labels)

In [259]:
model_7_results

{'accuracy': 82.80839895013123,
 'precision': 0.829756381651694,
 'recall': 0.8280839895013123,
 'f1': 0.8285067718575684}

In [272]:
### NOTE: Making splits like this will lead to data leakage ###
### (some of the training examples in the validation set) ###

### WRONG WAY TO MAKE SPLITS (train_df_shuffled has already been split) ###

# # Create subsets of 10% of the training data
# train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()
# len(train_sentences_10_percent), len(train_labels_10_percent)

In [ ]:
# One kind of correct way (there are more) to make data subset
# (split the already split train_sentences/train_labels)
train_sentences_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_sentences),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.1,
